### импортируем все нужное

In [119]:
import re
import pandas as pd
import csv
from tqdm import tqdm

In [120]:
try:
    from redditscore.tokenizer_rus import CrazyTokenizer as RusCrazyTokenizer
except:
    print('положите в папку библы redditscore файл tokenizer_rus.py (рядом с обычным tokenizer.py)')

from redditscore.tokenizer import CrazyTokenizer as EngCrazyTokenizer

In [121]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [122]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")

## Расширяем список стопслов

In [123]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/egor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [124]:
stops = stopwords.words('russian')
stops.extend(stopwords.words('english'))

In [125]:
badwords = [
'я', 'а', 'да', 'но', 'тебе', 'мне', 'ты', 'и', 'у', 'на', 'ага',
'так', 'там', 'какие', 'который', 'какая', 'туда', 'давай', 'короче', 'кажется', 'вообще',
'ну', 'чет', 'неа', 'свои', 'наше', 'хотя', 'такое', 'например', 'кароч', 'как-то',
'нам', 'хм', 'всем', 'нет', 'да', 'оно', 'своем', 'про', 'вы', 'тд', 'тп', 'т.д', 'т.п',
'вся', 'вам', 'это', 'эта', 'эти', 'этот', 'прям', 'либо', 'как', 'мы',
'просто', 'блин', 'очень', 'самые', 'твоем', 'ваша', 'кстати', 'вроде', 'типа', 'пока', 'ок',
    '°', '͜ʖ', '͡', 'ツ', 'ʕ•ᴥ•ʔ'
]

In [126]:
stops.extend(badwords)
stops.remove('не')
stops.remove('нет')
stops.remove('есть')

In [ ]:
print(stops)

## Расширяем список мер

чтоб не лезть руками в код спейси делаем чуть более костыльно

In [128]:
from spacy.lang.char_classes import UNITS
units = UNITS.split('|')
for u in 'кг|г|мг|мкг|л|мл|гр'.split('|'):
    if u not in units:
        units.append(u)
        
print(units)

['km', 'km²', 'km³', 'm', 'm²', 'm³', 'dm', 'dm²', 'dm³', 'cm', 'cm²', 'cm³', 'mm', 'mm²', 'mm³', 'ha', 'µm', 'nm', 'yd', 'in', 'ft', 'kg', 'g', 'mg', 'µg', 't', 'lb', 'oz', 'm/s', 'km/h', 'kmh', 'mph', 'hPa', 'Pa', 'mbar', 'mb', 'MB', 'kb', 'KB', 'gb', 'GB', 'tb', 'TB', 'T', 'G', 'M', 'K', '%', 'км', 'км²', 'км³', 'м', 'м²', 'м³', 'дм', 'дм²', 'дм³', 'см', 'см²', 'см³', 'мм', 'мм²', 'мм³', 'нм', 'кг', 'г', 'мг', 'м/с', 'км/ч', 'кПа', 'Па', 'мбар', 'Кб', 'КБ', 'кб', 'Мб', 'МБ', 'мб', 'Гб', 'ГБ', 'гб', 'Тб', 'ТБ', 'тбكم', 'كم²', 'كم³', 'م', 'م²', 'م³', 'سم', 'سم²', 'سم³', 'مم', 'مم²', 'مم³', 'كم', 'غرام', 'جرام', 'جم', 'كغ', 'ملغ', 'كوب', 'اكواب', 'мкг', 'л', 'мл', 'гр']


### Открываем данные

In [174]:
with open('/Users/egor/Desktop/HLPL/data_notprep/data_no3s/everyone_output_no3s.tsv', 'r', encoding='utf-8') as f:
    df = pd.read_csv(f, sep="\t")

In [175]:
df.head(5)

,tweet_id,class,plus,minus,all
0,persen_0.09400017398921579,0,Является эффективным препаратом выбора при куп...,Нельзя использовать в качестве монотерапии при...,Является эффективным препаратом выбора при куп...
1,mydocalm_0.8915672254450451,0,Является эффективным препаратом выбора в невро...,Может быть индивидуальная непереносимость.,Является эффективным препаратом выбора в невро...
2,flebodia-600_0.23622974547386466,0,Является хорошим венотоником. Используется в н...,Необходимы длительные курсы.,Является хорошим венотоником. Используется в н...
3,aphobazolum_0.7991345491221374,0,Является препаратом выбора при купировании тре...,"Замедленный эффект, к концу 2 недели приёма.",Является препаратом выбора при купировании тре...
4,zyllt_0.3909648947933738,1,"Я убеждён, что у пациентов, ожидающих чрезкожн...","Несомненно, что беспокоят малые кровотечения, ...","Я убеждён, что у пациентов, ожидающих чрезкожн..."


### выбираем тестовые твиты

In [132]:
test_data = [df['all'][8],
             df['all'][9], # дефисы
             df['all'][19],
             df['all'][103], # дефисы, капс
             
             df['all'][273], # латиница, нет пробелов вокруг скобки
             
             df['all'][10], # 0,5г ; р/д
             df['all'][20], # 125 мгк 2 раза ; Вентолин 100 мгк
            
             df['all'][100], # нет пробела между предложениями
             
             df['all'][14], # кавычки, юзернейм, эмоджи
             df['all'][18], # хештеги, юзернейм
             df['all'][25], # смайл, юзернейм, эмоджи
             df['all'][116], # эмоджи
             df['all'][102], # смайл
             
             df['all'][101] # сокращенная ссылка
            ]

In [133]:
#for tweet in test_data:
#    print(tweet)

## Настраиваем токенизатор

тут версия на улучшенном русском спейси [отсюда](https://github.com/aatimofeev/spacy_russian_tokenizer)

In [134]:
pos_emojis=['😀', '😁', '😂', '🤣', '😃', '😄', '😆', '😊', '😋', '😎', '😍', '😘', '🥰', '😚', '☺️', '🙂', '🤗', '🤩', '😏', '😌', '😛', '😜', '😝', '🙃', '🤑', '🤠', '😇', '🥳', '😈', '😺', '😸', '😹', '😻', '😽', '👍', '✌️', '🤟', '🤘', '👌', ' 🤙', '❤️', '🧡', '💛', '💚', '💙', '💜', '🖤', '❣️', '💕', '💞', '💓', '💗', '💖', '💘', '💝', '💟', '♥️', '🎊', '🎉', '🔝', '✔️', '🆒','☑️', '💑', '👩‍❤️‍👩', '👨‍❤️‍👨', '💏', '👩‍❤️‍💋‍👩', '👨‍❤️‍💋‍👨', '☀️', '🎆', '🌅', '🌄', '🌠', '🎇', '🎆', '🌇', '🌌', '💡', '🏖', '💯', '✅', '🎁', '💪', '👼']
neg_emojis=['🤨', '😐', '😑', '😶', '🙄', '😣', '😥', '🤐', '😫', '😓', '😔', '😕', '☹️', '🙁', '😖', '😞', '😟', '😤', '😢', '😭', '😦', '😧', '😨', '😩', '🤯', '😬', '😰', '😱', '😡', '😠', '🤬', '🤥', '👿', '😾', '😿', '🙀', '👎', '🤦‍♀️', '🤦‍♂️', '🙎‍♀️', '🙎‍♂️', '🙍‍♀️', '🙍‍♂️', '🥀', '🌥', '🌧', '⛈', '💔', '💢', '🔴', '🚨', '🧯', '⁉️']
neutral_emojis=['🤤', '😴', '🤒', '🌡️', '🤢', '😪', '🤮', '🤕', '💉', '💊', '😷', '😵', '💫', '🚑', '🦠', '🏥', '🤧', '🩺', '👨‍⚕️', '👩‍⚕️', '⚕️', '🧪', '🥵',  '🥶', '🥴', '💩', '🧟‍♀️', '🦴', '🦷', '🚽', '🦻', '👨‍🦯', '👩‍🦯', '👨‍🦼', '👩‍🦼', '👨‍🦽', ' 👩‍🦽', '🦼', '🦽', '🦯', '🩹', '♿️', '🚻', '🧫', '🧟‍♂️', '💤']


In [135]:
tokenizer_rus = RusCrazyTokenizer(lowercase=True, # к нижнему регистру
                           keepcaps=True, # все кроме all капс
                           urls='ЮРЛ', # удаляем ссылки
                           twitter_handles='ЮЗЕРНЕЙМ', # заменяем все юзернеймы
                           hashtags='ХЕШТЕГ', # пытаемся делить хештеги на слова (тут надо свой частотник закинуть) НЕ РЕАЛИЗОВАНО
                           pos_emojis=pos_emojis, neg_emojis=neg_emojis, neutral_emojis=neutral_emojis,
                           normalize=3, # удаляем больше 3 одинаковых знаков подряд
                           remove_punct=False, # удаляем пунктуацию
                           remove_breaks=True, # удаляем переносы строк
                           ignore_stopwords=stops, # удаляет стоп слова (разобраться почему падает на кастомном списке)
                           numbers = False,
                           extra_patterns = False
                          )

это на оригинальном английском (если не запускается русская)

In [136]:
tokenizer_eng = EngCrazyTokenizer(lowercase=True, # к нижнему регистру
                           keepcaps=True, # все кроме all капс
                           urls='ЮРЛ', # удаляем ссылки
                           twitter_handles='ЮЗЕРНЕЙМ', # заменяем все юзернеймы
                           hashtags='ХЕШТЕГ', # пытаемся делить хештеги на слова (тут надо свой частотник закинуть)
                           pos_emojis=True, neg_emojis=True, neutral_emojis=True, # меняем эмоджи на плейсхолдеры (кастомизировать)
                           normalize=3, # удаляем больше 3 одинаковых знаков подряд
                           remove_punct=False, # удаляем пунктуацию
                           remove_breaks=True, # удаляем переносы строк
                           ignore_stopwords=stops, # удаляет стоп слова (разобраться почему падает на кастомном списке)
                           numbers = False,
                           extra_patterns = False
                          )

In [ ]:
#vocab_rus = list(tokenizer_rus._nlp.vocab.strings)
#vocab_eng = list(tokenizer_eng._nlp.vocab.strings)

## Добавляем еще несколько удалений и замен

In [161]:
def process(tweet, log = False, lang = 'rus') -> str:
    tweet = re.sub('(\w)\.([А-ЯЁ])', r'\1. \2', tweet) # ставим пробелы
    tweet = re.sub('-', ' ', tweet) # меняем деффисы на пробел
    if log:
        #print('!до токенизации')
        print(tweet)
    # токенизируем (см. удаления и замены выше)
    if lang == 'rus':
        tokens = tokenizer_rus.tokenize(tweet)
    elif lang == 'eng':
        tokens = tokenizer_eng.tokenize(tweet)
    else:
        raise KeyError('ты долбаеб? языка {} нет'.format(lang))
        
    if log:
        #print('!после токенизации')
        print(tokens)
        
    new_tokens = []
    for token in tokens:
        if re.search('\d+', token):
            try:
                x = re.findall('[^\d.,\s]+', token)[0]
                if x in units:
                    token = 'NUM MEASURE'
                else:
                    continue
            except:
                token = 'NUM'
        #if re.search('[!"#$%&\'()*+/:;<=>?@[\]^_`{|}~]', token): #удаляем оставшиеся символы
            #token = re.sub('[!"#$%&\'()*+/:;<=>?@[\]^_`{|}~]', '', token)
        
        if token in units:
            token = 'MEASURE'
        
        if 'ё' in token:
            token = token.replace('ё', 'е')
        
                
        if len(token) < 2 and not re.search('[,\.!"#$%&\'()*+/:;<=>?@[\]^_`{|}~]', token):
            continue
           
        new_tokens.append(token)
            
    if log:
        #print('!после обработки каждого токена')
        print(new_tokens)
        #print('!output')
    
    return ' '.join(new_tokens)

In [162]:
res = []
for tweet in test_data:
    print(tweet)
    tweet = process(str(tweet), log=False, lang='rus')
    res.append(tweet)
    print(tweet, '\n')

 Средство очень эффективное, основным действующим веществом является  Нурофен не только избавляет от боли, но и помогает снять воспаление десневой ткани.   Противопоказаниями служат болезни почек, печени, гипертензия, болезнь Крона. 
средство эффективное , основным действующим веществом является нурофен не избавляет боли , помогает снять воспаление десневой ткани . противопоказаниями служат болезни почек , печени , гипертензия , болезнь крона . 

- Препарат эффективный при лечении акне, не заставляет долго ждать результата. Так как не обладает фотосенсибилизацией актуален для применения в южных регионах нашей страны. Обладает отбеливающим свойством, что немаловажно при коррекции заболевания. Огромный плюс - разрешен при беременности и лактации. Дает раздражение кожи на начальных этапах лечения, но при правильно подобраной дозировка и кратности применения этого можно избежать.
препарат эффективный лечении акне , не заставляет долго ждать результата . не обладает фотосенсибилизацией акту

In [ ]:
res

## тут можно тестировать конкретные случаи и писать о  траблах ане

In [ ]:
test_phrase = 'я вдолбил 100г мефа и 20мл пива 50л колы и 18кг моркови а еще потом занюхал 18мг герыча'
log = False
lang = 'rus'

In [ ]:
test_tokenizer = EngCrazyTokenizer(lowercase=True, # к нижнему регистру
                           keepcaps=True, # все кроме all капс
                           urls='ЮРЛ', # удаляем ссылки
                           twitter_handles='ЮЗЕРНЕЙМ', # заменяем все юзернеймы
                           hashtags='ХЕШТЕГ', # пытаемся делить хештеги на слова (тут надо свой частотник закинуть)
                           pos_emojis=False, neg_emojis=False, neutral_emojis=False, # меняем эмоджи на плейсхолдеры (кастомизировать)
                           normalize=3, # удаляем больше 3 одинаковых знаков подряд
                           remove_punct=False, # удаляем пунктуацию
                           remove_breaks=True, # удаляем переносы строк
                           ignore_stopwords=False, # удаляет стоп слова (разобраться почему падает на кастомном списке)
                           numbers = False,
                           extra_patterns = False
                          )

In [ ]:
print('before  :', test_phrase)
res_phrase = process(test_phrase, log = log, lang = lang)
print('after   :',res_phrase)

print('del/repl:', ', '.join([w for w in test_tokenizer.tokenize(test_phrase) if w not in res_phrase.split(' ')]))

## Лемматизация

### nltk snowball stemmer

In [ ]:
for tweet in res:
    print(tweet)
    print(' '.join([stemmer.stem(w) for w in tweet.split(' ')]), '\n')

### Mystem

In [140]:
from pymystem3 import Mystem
mstm = Mystem()

In [141]:
lem = []
for tweet in tqdm(res):
    lem.append(''.join(mstm.lemmatize(tweet)).strip())

100%|██████████| 14/14 [00:01<00:00,  9.42it/s]


In [ ]:
lem

### pymorphy

In [143]:
lem2 = []
for tweet in tqdm(res):
    lem_tweet = []
    for word in tweet.split():
        lem_tweet.append(morph.parse(word)[0].normal_form)
    lem2.append(' '.join(lem_tweet))

100%|██████████| 14/14 [00:00<00:00, 62.48it/s]


In [ ]:
lem2

In [ ]:
for t1, t2 in zip(lem, lem2):
    for w1, w2 in zip(t1.split(), t2.split()):
        if w1 != w2:
            print(w1, w2)

## Соединение train и val

In [ ]:
with open('../original_data/task2_ru_training.tsv', 'r', encoding='utf-8') as tr:
    with open('../original_data/task2_ru_validation.tsv', 'r', encoding='utf-8') as val:
        
        df1 = pd.read_csv(tr, sep="\t")
        df2 = pd.read_csv(val, sep="\t")

In [ ]:
combined = pd.concat([df1, df2])

In [ ]:
combined

### Открываем данные

In [191]:
with open('/Users/egor/Desktop/HLPL/data_notprep/task2_ru_test_participant.tsv', 'r', encoding='utf-8') as f:
    df = pd.read_csv(f, sep="\t")

In [192]:
df.head(5)

,tweet_id,tweet
0,892079521922416641,@A_Kapustin запретить на хрен.. недосмотр одн...
1,1089927935031676929,"не тратьте деньги на образование, тратьте на т..."
2,882953471774904321,а туда лазить нехуй. ваша хандра по несчастным...
3,1053255070094184448,"Да, думаю, должен подойти. Хотя я порой лежу и..."
4,952310182461300736,"по идеи, людям прописывают до 4х таблеток кети..."


# Полный скрипт препроцессинга

In [193]:
def write_data(df, out_file, prep = 'tweet', lem = 'prep_tweet', stem = 'lem_tweet'):
    
    if prep:
        new_tweets = []
        for tweet in tqdm(df[prep]):
            new_tweets.append(process(tweet, log=False, lang = 'rus'))
        df['prep_tweet'] = new_tweets
        
        
#     if lemmatize == 'mystem':
#         lem = []
#         for tweet in tqdm(df[prep_tweets]):
#             lem.append(''.join(mstm.lemmatize(tweet)).strip())
#         df['lem_tweet'] = lem

    if lem:
        lems = []
        for tweet in tqdm(df[lem]):
            lem_tweet = []
            for word in tweet.split():
                lem_tweet.append(morph.parse(word)[0].normal_form)
            lems.append(' '.join(lem_tweet))
        df['lem_tweet'] = lems
        
    
    if stem:
        stems = []
        for tweet in tqdm(df[stem]):
            stems.append(' '.join([stemmer.stem(w).strip() for w in tweet.split(' ')]))
        df['stem_tweet'] = stems


    with open('/Users/egor/Desktop/HLPL/{}'.format(out_file), 'w', encoding='utf-8') as output:
        df.to_csv(output, sep='\t', index=False, line_terminator='\n')

    print('Done! Your data is in {}'.format(out_file))

In [194]:
out_file = 'task2_ru_test_participant_prep_lem_stem_withpunct.tsv'

In [195]:
#df = файл с импорта давно
write_data(df, out_file,
           prep = 'tweet',
           lem = 'prep_tweet',
           stem = 'lem_tweet'
          )

100%|██████████| 1903/1903 [00:07<00:00, 268.03it/s]


Done! Your data is in task2_ru_test_participant_prep_lem_stem_withpunct.tsv


## Проверка частотности: схожа ли частотность в протаблетках с изначальной? 
Если не схожа, то надо редактировать. Например, удалить слово побочка.

In [ ]:
with open('/Users/egor/Desktop/HLPL/additional_egor_minus+all_prep_lem_stem_withpunct.tsv', 'r', encoding='utf-8') as f:
    df = pd.read_csv(f, sep="\t")

In [ ]:
from collections import Counter


new_df = df[['stem_minus','stem_all']].copy()


stem_minus_string = ''
stem_all_string = ''
for entry in new_df['stem_minus']:
    stem_minus_string += str(entry) + ' '
for entry in new_df['stem_all']:
    stem_all_string += str(entry) + ' '
    
    
stem_minus_string = re.sub(r'[^\w\s]','', stem_minus_string)
stem_all_string = re.sub(r'[^\w\s]','', stem_all_string)


stem_minus_freqlist = Counter(stem_minus_string.split()).most_common()
stem_all_freqlist = Counter(stem_all_string.split()).most_common()

In [ ]:
with open('/Users/egor/Desktop/HLPL/combined_prep_lem_stem_withpunct.tsv', 'r', encoding='utf-8') as f:
    df_orig = pd.read_csv(f, sep="\t")

In [ ]:
new_df_orig = df_orig[['stem_tweet']].copy()

stem_tweet_string = ''
for entry in new_df_orig['stem_tweet']:
    stem_tweet_string += str(entry) + ' '
stem_tweet_string = re.sub(r'[^\w\s]','', stem_tweet_string)
stem_tweet_freqlist = Counter(stem_tweet_string.split()).most_common()

In [ ]:
stem_minus_freqlist

In [ ]:
stem_all_freqlist

In [ ]:
stem_tweet_freqlist

## Считаем частотность в долях

In [118]:
[print((i, Counter(stem_all_string.split())[i] / len(stem_all_string.split()) * 100.0)) for i, count in stem_all_freqlist]

('препарат', 4.358360029073681)
('не', 2.6139392144722318)
('эффект', 1.6205884728241848)
('num', 1.3783078041295393)
('эффективн', 1.2760115217918002)
('лечен', 1.2679354995019785)
('пациент', 1.0821869868360836)
('хорош', 0.9395105930492368)
('применен', 0.9072065038899507)
('цен', 0.8749024147306648)
('побочн', 0.7591460952432229)
('использова', 0.7241499986539963)
('при', 0.7026139392144722)
('действ', 0.6541578054755431)
('назнача', 0.573397582577328)
('практик', 0.5518615231378038)
('возможн', 0.5276334562683392)
('применя', 0.5249414488383988)
('высок', 0.5007133819689342)
('дан', 0.4711013002395886)
('част', 0.46302527794976717)
('средств', 0.4549492556599456)
('форм', 0.4468732333701241)
('рекомендова', 0.43072118879048105)
('терап', 0.4226451665006596)
('достаточн', 0.419953159070719)
('сво', 0.414569144210838)
('качеств', 0.409185129350957)
('ребенок', 0.409185129350957)
('длительн', 0.4064931219210165)
('заболеван', 0.39034107734137347)
('кож', 0.38226505505155195)
('быстр'

('единствен', 0.06460817831857216)
('количеств', 0.06460817831857216)
('риск', 0.06460817831857216)
('отм', 0.06460817831857216)
('однак', 0.06460817831857216)
('завыс', 0.06460817831857216)
('вирусн', 0.06460817831857216)
('антибиотик', 0.06460817831857216)
('гипертенз', 0.061916170888631654)
('веществ', 0.061916170888631654)
('печен', 0.061916170888631654)
('провер', 0.061916170888631654)
('незаменим', 0.061916170888631654)
('упаковк', 0.061916170888631654)
('сраз', 0.061916170888631654)
('отношен', 0.061916170888631654)
('дела', 0.061916170888631654)
('повыша', 0.061916170888631654)
('разн', 0.061916170888631654)
('помоч', 0.061916170888631654)
('нрав', 0.061916170888631654)
('горазд', 0.061916170888631654)
('консультац', 0.061916170888631654)
('проконсультирова', 0.061916170888631654)
('качествен', 0.061916170888631654)
('обычн', 0.061916170888631654)
('желез', 0.061916170888631654)
('следова', 0.059224163458691144)
('проявлен', 0.059224163458691144)
('доказательн', 0.0592241634586

('времен', 0.03499609658922659)
('иммунитет', 0.03499609658922659)
('стад', 0.03499609658922659)
('чувств', 0.03499609658922659)
('лиш', 0.03499609658922659)
('профилактическ', 0.03499609658922659)
('растительн', 0.03499609658922659)
('вирус', 0.03499609658922659)
('хейлит', 0.03499609658922659)
('поврежден', 0.03499609658922659)
('комплекс', 0.03230408915928608)
('последн', 0.03230408915928608)
('противовирусн', 0.03230408915928608)
('остава', 0.03230408915928608)
('стандарт', 0.03230408915928608)
('ожида', 0.03230408915928608)
('слаб', 0.03230408915928608)
('минимальн', 0.03230408915928608)
('противосудорожн', 0.03230408915928608)
('вниман', 0.03230408915928608)
('дозирован', 0.03230408915928608)
('достига', 0.03230408915928608)
('флакон', 0.03230408915928608)
('уролог', 0.03230408915928608)
('гиперем', 0.03230408915928608)
('минут', 0.03230408915928608)
('процедур', 0.03230408915928608)
('концентрац', 0.03230408915928608)
('язв', 0.03230408915928608)
('сопутств', 0.03230408915928608

('успех', 0.021536059439524052)
('прийт', 0.021536059439524052)
('решен', 0.021536059439524052)
('рад', 0.021536059439524052)
('капл', 0.021536059439524052)
('жалоб', 0.021536059439524052)
('забыва', 0.021536059439524052)
('стабильн', 0.021536059439524052)
('наступлен', 0.021536059439524052)
('город', 0.021536059439524052)
('ангин', 0.021536059439524052)
('бронх', 0.021536059439524052)
('депресс', 0.021536059439524052)
('немног', 0.021536059439524052)
('остальн', 0.021536059439524052)
('повторн', 0.021536059439524052)
('идеальн', 0.021536059439524052)
('трудн', 0.021536059439524052)
('переход', 0.021536059439524052)
('характер', 0.021536059439524052)
('определен', 0.021536059439524052)
('неоднократн', 0.021536059439524052)
('гормональн', 0.021536059439524052)
('кишечник', 0.021536059439524052)
('кератолитическ', 0.021536059439524052)
('подобн', 0.021536059439524052)
('мнен', 0.021536059439524052)
('старш', 0.021536059439524052)
('появля', 0.021536059439524052)
('токсичн', 0.02153605943

('ин', 0.01615204457964304)
('раздражительн', 0.01615204457964304)
('клетк', 0.01615204457964304)
('увлека', 0.01615204457964304)
('взят', 0.01615204457964304)
('составля', 0.01615204457964304)
('фенибут', 0.01615204457964304)
('натуральн', 0.01615204457964304)
('гепат', 0.01615204457964304)
('диспепсическ', 0.01615204457964304)
('аффект', 0.01615204457964304)
('вобэнзим', 0.01615204457964304)
('пожал', 0.01615204457964304)
('кто', 0.01615204457964304)
('вер', 0.01615204457964304)
('повод', 0.01615204457964304)
('гриб', 0.01615204457964304)
('универсальн', 0.01615204457964304)
('муколитическ', 0.01615204457964304)
('разогрева', 0.01615204457964304)
('способ', 0.01615204457964304)
('образ', 0.01615204457964304)
('верхн', 0.01615204457964304)
('невроз', 0.01615204457964304)
('каптопр', 0.01615204457964304)
('эпизодическ', 0.013460037149702533)
('дополнен', 0.013460037149702533)
('психологическ', 0.013460037149702533)
('проч', 0.013460037149702533)
('доверя', 0.013460037149702533)
('совер

...

('синуфорт', 0.0026920074299405065)
('профильн', 0.0026920074299405065)
('свертываем', 0.0026920074299405065)
('жк', 0.0026920074299405065)
('гипертерм', 0.0026920074299405065)
('вазотенз', 0.0026920074299405065)
('секреторн', 0.0026920074299405065)
('гортаноглотк', 0.0026920074299405065)
('исчеза', 0.0026920074299405065)
('фотосенсибилизир', 0.0026920074299405065)
('скинор', 0.0026920074299405065)
('максимум', 0.0026920074299405065)
('разлета', 0.0026920074299405065)
('артериол', 0.0026920074299405065)
('демократичн', 0.0026920074299405065)
('свест', 0.0026920074299405065)
('энмг', 0.0026920074299405065)
('белков', 0.0026920074299405065)
('пересушиван', 0.0026920074299405065)
('лихорадк', 0.0026920074299405065)
('школьн', 0.0026920074299405065)
('передава', 0.0026920074299405065)
('тинидазол', 0.0026920074299405065)
('сбит', 0.0026920074299405065)
('озноб', 0.0026920074299405065)
('потовыведен', 0.0026920074299405065)
('попотет', 0.0026920074299405065)
('старт', 0.0026920074299405065)

('ляминдискэктом', 0.0026920074299405065)
('комплаенс', 0.0026920074299405065)
('выборочн', 0.0026920074299405065)
('откуд', 0.0026920074299405065)
('материнск', 0.0026920074299405065)
('css', 0.0026920074299405065)
('плазм', 0.0026920074299405065)
('прайс', 0.0026920074299405065)
('недигидропиридин', 0.0026920074299405065)
('аг', 0.0026920074299405065)
('облепихов', 0.0026920074299405065)
('загрязня', 0.0026920074299405065)
('грамположительн', 0.0026920074299405065)
('затрудн', 0.0026920074299405065)
('полипрагмаз', 0.0026920074299405065)
('обывател', 0.0026920074299405065)
('всасыван', 0.0026920074299405065)
('перчатк', 0.0026920074299405065)
('полден', 0.0026920074299405065)
('джвп', 0.0026920074299405065)
('иммуносупрессивн', 0.0026920074299405065)
('проглатыва', 0.0026920074299405065)
('амигренин', 0.0026920074299405065)
('распорядок', 0.0026920074299405065)
('латексн', 0.0026920074299405065)
('дгу', 0.0026920074299405065)
('диспепс', 0.0026920074299405065)
('потребля', 0.00269200

('нейтральн', 0.0026920074299405065)
('ослабева', 0.0026920074299405065)
('предшествова', 0.0026920074299405065)
('хроносфер', 0.0026920074299405065)
('формул', 0.0026920074299405065)
('аэп', 0.0026920074299405065)
('затянут', 0.0026920074299405065)
('периодонтальн', 0.0026920074299405065)
('перианальн', 0.0026920074299405065)
('обновля', 0.0026920074299405065)
('воспоминан', 0.0026920074299405065)
('кипферон', 0.0026920074299405065)
('интерферонов', 0.0026920074299405065)
('косвен', 0.0026920074299405065)
('переноск', 0.0026920074299405065)
('знак', 0.0026920074299405065)
('жеч', 0.0026920074299405065)
('нервозн', 0.0026920074299405065)
('бодр', 0.0026920074299405065)
('отойт', 0.0026920074299405065)
('римантадин', 0.0026920074299405065)
('rimantadine', 0.0026920074299405065)
('хпн', 0.0026920074299405065)
('спровоцирова', 0.0026920074299405065)
('скачк', 0.0026920074299405065)
('геморрагическ', 0.0026920074299405065)
('бригад', 0.0026920074299405065)
('рсутк', 0.0026920074299405065)


('нейролептик', 0.0026920074299405065)
('дереализац', 0.0026920074299405065)
('вышеперечислен', 0.0026920074299405065)
('мокнут', 0.0026920074299405065)
('прет', 0.0026920074299405065)
('буллезн', 0.0026920074299405065)
('эритразм', 0.0026920074299405065)
('интертригинозн', 0.0026920074299405065)
('подсуш', 0.0026920074299405065)
('присоединя', 0.0026920074299405065)
('грибок', 0.0026920074299405065)
('мелкоточечн', 0.0026920074299405065)
('кератолизис', 0.0026920074299405065)
('подошв', 0.0026920074299405065)
('баночк', 0.0026920074299405065)
('эффекет', 0.0026920074299405065)
('вич', 0.0026920074299405065)
('угрет', 0.0026920074299405065)
('прерва', 0.0026920074299405065)
('волшебн', 0.0026920074299405065)
('настр', 0.0026920074299405065)
('краснодар', 0.0026920074299405065)
('присоединен', 0.0026920074299405065)
('статус', 0.0026920074299405065)
('женск', 0.0026920074299405065)
('менструац', 0.0026920074299405065)
('клензит', 0.0026920074299405065)
('клиндомицин', 0.0026920074299405

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,